# Reddit: 'Front Page of the Internet'

![](./images/reddit_search_result.png)

## What does it take for a post to get there?
![](./images/reddit_top.png)

# Big Query June 2017 Data for 'default' subreddits

![](./images/bg_table_desc.png)

# Getting Going with Spark

In [1]:
import findspark
# my local spark install
findspark.init('/Users/dreyco676/spark-2.2.0-bin-hadoop2.7/')

In [2]:
import pyspark
from pyspark.sql import SQLContext

# create spark contexts
sc = pyspark.SparkContext()
sqlContext = SQLContext(sc)

### Here we can import a CSV to a spark dataframe

In [3]:
df = sqlContext.read.csv('data/reddit_defaults_june17.csv', header=True, inferSchema=True)

In [4]:
df.count()

1087311

## Know your data
Data Science can't be done in a vacuum. You need to know what your data represents.

https://github.com/reddit/reddit/wiki/JSON

## Multiple fields are not needed.
* `saved` is only applicable to the person that pulled the data

* `id` is a duplicate of the `author` field

* `thumbnail` is a randomly assigned reddit link for posts that have a link

* `subreddit_id` is a duplicate of `subreddit` field


In [5]:
drop_cols = ['saved', 'id', 'thumbnail', 'subreddit_id']
for col in drop_cols:
    df.drop(col)

## Some records are not valid.
Sometimes users delete their posts, filter them out

In [6]:
df = df.filter(df["title"] != "[deleted]")

In [7]:
df.count()

945380

In [8]:
df.dtypes

[('created_utc', 'string'),
 ('subreddit', 'string'),
 ('author', 'string'),
 ('domain', 'string'),
 ('url', 'string'),
 ('num_comments', 'string'),
 ('score', 'string'),
 ('ups', 'string'),
 ('downs', 'string'),
 ('title', 'string'),
 ('selftext', 'string'),
 ('saved', 'string'),
 ('id', 'string'),
 ('from_kind', 'string'),
 ('gilded', 'string'),
 ('from', 'string'),
 ('stickied', 'string'),
 ('retrieved_on', 'string'),
 ('over_18', 'string'),
 ('thumbnail', 'string'),
 ('subreddit_id', 'string'),
 ('hide_score', 'string'),
 ('link_flair_css_class', 'string'),
 ('author_flair_css_class', 'string'),
 ('archived', 'string'),
 ('is_self', 'string'),
 ('from_id', 'string'),
 ('permalink', 'string'),
 ('name', 'string'),
 ('author_flair_text', 'string'),
 ('quarantine', 'string'),
 ('link_flair_text', 'string'),
 ('distinguished', 'string')]

# ಠ_ಠ
### Even though we told it to infer the schema it still set everything to string.

In [9]:
int_cols = ['num_comments', 'score', 'ups', 'downs', 'over_18']
bool_cols = ['gilded', 'stickied', 'hide_score', 'archived', 'is_self', 'quarantine']
date_cols = ['created_utc', 'retrieved_on']

In [10]:
df[['created_utc']].show(5)

+-----------+
|created_utc|
+-----------+
| 1497045651|
| 1498078666|
| 1498318066|
| 1497534495|
| 1498002534|
+-----------+
only showing top 5 rows



## Our dates are in Unix Epoch time 
(seconds since 1970-01-01)

In [11]:
from pyspark.sql.functions import from_unixtime

for col in date_cols:
    df = df.withColumn(col, from_unixtime(df[col]))

In [12]:
df[['created_utc']].show(5)

+-------------------+
|        created_utc|
+-------------------+
|2017-06-09 17:00:51|
|2017-06-21 15:57:46|
|2017-06-24 10:27:46|
|2017-06-15 08:48:15|
|2017-06-20 18:48:54|
+-------------------+
only showing top 5 rows



## Convert Strings to Integers
(never cast things you wouldn't do math on, ie IDs, phone numbers, zip codes, etc)

In [13]:
from pyspark.sql.types import IntegerType

for col in int_cols:
    df = df.withColumn(col, df[col].cast(IntegerType()))

## Convert Strings to Booleans
(Yes/No, 1/0, True/False)

In [14]:
from pyspark.sql.types import BooleanType

for col in bool_cols:
    df = df.withColumn(col, df[col].cast(BooleanType()))

In [15]:
df.dtypes

[('created_utc', 'string'),
 ('subreddit', 'string'),
 ('author', 'string'),
 ('domain', 'string'),
 ('url', 'string'),
 ('num_comments', 'int'),
 ('score', 'int'),
 ('ups', 'int'),
 ('downs', 'int'),
 ('title', 'string'),
 ('selftext', 'string'),
 ('saved', 'string'),
 ('id', 'string'),
 ('from_kind', 'string'),
 ('gilded', 'boolean'),
 ('from', 'string'),
 ('stickied', 'boolean'),
 ('retrieved_on', 'string'),
 ('over_18', 'int'),
 ('thumbnail', 'string'),
 ('subreddit_id', 'string'),
 ('hide_score', 'boolean'),
 ('link_flair_css_class', 'string'),
 ('author_flair_css_class', 'string'),
 ('archived', 'boolean'),
 ('is_self', 'boolean'),
 ('from_id', 'string'),
 ('permalink', 'string'),
 ('name', 'string'),
 ('author_flair_text', 'string'),
 ('quarantine', 'boolean'),
 ('link_flair_text', 'string'),
 ('distinguished', 'string')]

In [16]:
# Use langid module to classify the language to make sure we are applying the correct cleanup actions for English
# https://github.com/saffsd/langid.py
import langid

def check_lang(data_str):
    language = 'NA'
    if data_str is not None:
        predict_lang = langid.classify(data_str)
        if predict_lang[1] >= .7:
            language = predict_lang[0]
    return language

In [17]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

# Register all the functions in Preproc with Spark Context
check_lang_udf = udf(check_lang, StringType())

In [18]:
# predict language
df = df.withColumn("language", check_lang_udf(df["selftext"]))

In [19]:
df[['selftext', 'language']].show(20, truncate=20)

+--------------------+--------+
|            selftext|language|
+--------------------+--------+
|                null|      NA|
|              Marker|      en|
|                null|      NA|
|                null|      NA|
|                null|      NA|
|                null|      NA|
|                null|      NA|
|                null|      NA|
|           [deleted]|      en|
|                null|      NA|
|                null|      NA|
|                null|      NA|
|                null|      NA|
|                null|      NA|
|                null|      NA|
|                null|      NA|
|                null|      NA|
|           [deleted]|      en|
|                null|      NA|
| illustration art...|      NA|
+--------------------+--------+
only showing top 20 rows



created_utc --day of week, hour of day
subreddit
author
domain
url
num_comments
score
ups
downs
title
selftext
saved --data collection artifact
id --duplicate of author
from_kind
gilded
from
stickied
retrieved_on
over_18
thumbnail --info contained in is_self
subreddit_id --duplicate of subreddit
hide_score
link_flair_css_class
author_flair_css_class
archived
is_self
from_id
permalink
name
author_flair_text
quarantine
link_flair_text
distinguished